In [4]:
def distance(pitch_id, batter_id,balls,strikes, pitch_count, inning, c=10):
    def id_dis(id1, id2):
        return 0 if id1 == id2 else c
    def dis(x):
        sqdis = (x['balls']-balls)**2+\
        (x['strikes']-strikes)**2+\
        (x['pitch_count']-pitch_count)**2+\
        (x['inning']-inning)**2+\
        id_dis(pitch_id,x['pitcher'])+\
        id_dis(batter_id,x['batter'])
        return sqdis
    return dis
def predict(all_data,pitch_id, batter_id,balls,strikes, pitch_count,inning,
            num_of_neighbor=5, c=10):
    f_l=distance(pitch_id,batter_id,balls,strikes,pitch_count,inning,c=c)
    all_data["dis"]=all_data.apply(f_l,axis=1)
    sort_results=all_data.sort_values(by=["dis"])
    select_umpcall = sort_results[0:num_of_neighbor]["umpcall"]
    
    #count weighted states
    X = 0
    S = 0
    B = 0
    i = 0
    for state in select_umpcall:
        tmp = math.exp(-i/num_of_neighbor*3)
        if state == "X":
            X+=tmp
        elif state == "S":
            S+=tmp
        else:
            B+=tmp
        i+=1
    XBS = X+S+B
    print(sort_results[["dis","umpcall"]])
    return {"S": S/XBS,
            "B": B/XBS,
            "X": X/XBS}

In [5]:
f_l=distance("greinza01","spande01",1,1,3,1,c=10)
f["dis"]=f.apply(f_l,axis=1)

In [6]:
select_umpcall = f.sort_values(by=["dis"])[0:10]["umpcall"]
print(f.sort_values(by=["dis"])[0:10][["dis","umpcall"]])

        dis umpcall
2         0       X
564500    0       B
1         2       B
654957    2       S
564499    2       B
654958    2       B
654959    2       X
564501    2       S
0         6       S
564502    6       S


In [7]:
print(predict(f,"holdejo02","sanchga02",1,2,16,7,num_of_neighbor=7, c=10))

          dis umpcall
632583     10       B
512        10       X
159171     11       S
356268     11       S
148957     11       S
148958     11       S
159172     11       S
152286     11       S
356269     11       B
124121     11       S
632582     12       B
632584     12       B
511        12       B
462925     12       X
318223     12       S
695136     12       S
174540     12       X
124122     12       B
124120     12       S
503341     13       X
148956     13       B
221048     13       B
695135     13       S
152285     13       S
236379     13       S
159252     14       X
318222     14       S
327036     14       X
291370     14       S
266045     14       S
...       ...     ...
111828  11470       B
122527  11470       S
194812  11471       S
138352  11474       S
278992  11474       B
388081  11475       B
99671   11475       S
436560  11475       S
251079  11477       S
111829  11686       B
99672   11687       B
436561  11687       S
138353  11689       S
388082  11

# Experient 2

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
base_dir = "MLB_2017/"
f = pd.read_csv(base_dir+"MLB_PitchFX_2017_RegularSeason.csv")

In [3]:
f.columns.values
label2one = {'B':[1,0,0],'S':[0,1,0],'X':[0,0,1]}
one2label = {0:'B', 1:'S', 2:'X'}

In [4]:
def normalize(data):
    mu = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return mu, std, (data-mu)/std

In [5]:
train_x = f[['x0','x','y','ax','ay','az','px','pz','sz_top','sz_bot',
             'vx0','vy0','vz0','pfx_x','z0','start_speed','end_speed',
             'break_y','break_angle','break_length','spin_dir','spin_rate']]
train_y = f['umpcall']

data_x = train_x.as_matrix()
data_y = train_y.as_matrix()
from sklearn.utils import shuffle
data_x, data_y = shuffle(data_x, data_y)


In [6]:
from sklearn.model_selection import train_test_split
train_x, dev_x, train_y, dev_y = train_test_split(data_x, data_y, test_size=0.1, random_state=0)

In [7]:
np.mean(data_x, axis=0)

array([ -7.81418881e-01,   1.15070170e+02,   1.76193902e+02,
        -3.47046123e+00,   2.58193594e+01,  -2.19335740e+01,
         5.07249598e-02,   2.25897458e+00,   3.41283647e+00,
         1.54641061e+00,   2.77957433e+00,  -1.28703931e+02,
        -4.43331552e+00,  -1.81589433e+00,   5.76000239e+00,
         8.86837799e+01,   8.14302932e+01,   2.38269708e+01,
         8.63518099e+00,   6.68905396e+00,   1.84361395e+02,
         1.80968426e+03])

In [116]:
def predict(model,x, train_x, train_y):
    dist, indx = model.kneighbors(x)
    preds = []
    for jj in range(len(indx)):
        dis = {}
        ctr=0
        for i,e in enumerate(indx[jj]):
            ctr += 1/dist[jj]
            if train_y[e] not in dis:
                dis[train_y[e]] = 1/dist[jj][i]
            else:
                dis[train_y[e]] += 1/dist[jj][i]
        themax = -1
        thekey = []
        for k in dis.keys():
            if dis[k] > themax:
                themax = dis[k]
                thekey = k
        preds.append(thekey)
    return preds
        

In [11]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(n_neighbors=5, algorithm='auto', n_jobs=4)

In [13]:
model.fit(train_x)


KeyboardInterrupt: 

# Cross Validation

In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
class kNNClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, **kwargs):
        self.model = NearestNeighbors(**kwargs)
    def fit(self, X, y):
        self.mu, self.std, self.X = normalize(X)
        self.y = y
        self.model.fit(self.X)
        
    def predict(self, dev_x):
        dist, indx = self.model.kneighbors(dev_x)
        preds = []
        for jj in range(len(indx)):
            dis = {}
            ctr=0
            for i,e in enumerate(indx[jj]):
                ctr += 1/dist[jj][i]**2
                if self.y[e] not in dis:
                    dis[self.y[e]] = 1/dist[jj][i]**2
                else:
                    dis[self.y[e]] += 1/dist[jj][i]**2
            themax = -1
            thekey = []
            for k in dis.keys():
                if dis[k] > themax:
                    themax = dis[k]
                    thekey = k
            preds.append(thekey)
        return preds
    def score(self, X, y=None):
        pred = self.predict((X-self.mu)/self.std)
        if y is None:
            return 0
        return np.sum(pred == y) / len(y)

In [13]:
model = kNNClassifier(n_neighbors=20, algorithm='auto', n_jobs=4)

In [16]:
model.fit(train_x, train_y)

In [17]:
model.score(dev_x,dev_y)

0.69910836570981361

In [15]:
pred = model.predict(dev_x)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, data_x, data_y, cv=5)
scores

In [ ]:
from sklearn.model_selection import cross_val_score
model = kNNClassifier(n_neighbors=50, algorithm='auto', n_jobs=4)
scores = cross_val_score(model, data_x, data_y, cv=5)
scores

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10,10,), max_iter=200, alpha=1e-4,
                    solver='adam', tol=1e-4,
                    learning_rate_init=.001)
mlp.fit(train_x, train_y)

In [ ]:
np.sum(mlp.predict(dev_x)==dev_y)/len(dev_y)

In [16]:
len(dev_y)

71554

In [214]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,50,), max_iter=100, alpha=1e-4,
                    solver='adam', tol=1e-4,
                    learning_rate_init=.001)

scores = cross_val_score(mlp, data_x, data_y, cv=5)
scores

/Users/huangyinhsun/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 